In [1]:
import requests
import locale

from lxml import html
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
from pprint import pprint
from datetime import date

locale.setlocale(locale.LC_TIME, 'ru')


'ru'

In [2]:
# Подготовим базу данных
client = MongoClient('localhost', 27017)
db = client['news']  # hh - database
my_news = db.my_news  # items - collection



In [3]:
# функция добавления данных в БД
def add_data_to_db(doc, only_new = True):
    exists = my_news.find_one({'link': doc['link']})
    if only_new == True:
        if not exists:
            try:
                my_news.insert_one(doc)
            except DuplicateKeyError:
                pass
    else:
        try:
            my_news.insert_one(doc)
        except DuplicateKeyError:
            pass

In [4]:
# Получаем данные со страницы
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
url = "https://lenta.ru"
session = requests.Session()

response = session.get(url+'/parts/news', headers=headers)

# https://lenta.ru/parts/news/
print(f'URL is {response.url}')
print(f'Status code is {response.status_code}')

dom = html.fromstring(response.text)


URL is https://lenta.ru/parts/news
Status code is 200


In [5]:
items = dom.xpath("//li[@class='parts-page__item']")
# all_news = []
for item in items:
    news = {}
    resource = url
    name = item.xpath(".//h3[@class='card-full-news__title']/text()")
    link = item.xpath(".//h3[@class='card-full-news__title']/../@href")
    news_time = item.xpath(".//time[contains(@class,'card-full-news__date')]/text()")
    

    news['resource'] = resource
    news['name'] = ''.join(name)
    
    if 'https' in str(link):
        news['link'] = ''.join(link)
    else:
        news['link'] = url + ''.join(link)
    
    if len(''.join(news_time))<10:
        news['news_time'] = ''.join(news_time) + ', ' + date.today().strftime("%d %B %Y")
    else:
        news['news_time'] = ''.join(news_time)


    add_data_to_db(news, only_new=True)


In [6]:
# Проверяем, что данные в БД
for item in my_news.find({'resource': url}):
    pprint(item)

{'_id': ObjectId('64bcd2464155afe95155fed9'),
 'link': 'https://moslenta.ru/news/city/dojdi-i-grozi-23-07-2023.htm/',
 'name': 'Синоптик предупредил москвичей о дождях и грозах',
 'news_time': '10:03, 23 Июль 2023',
 'resource': 'https://lenta.ru'}
{'_id': ObjectId('64bcd2474155afe95155feda'),
 'link': 'https://lenta.ru/news/2023/07/23/kassetttt/',
 'name': 'Военные эксперты оценили ход наступления ВСУ после применения '
         'кассетных снарядов',
 'news_time': '10:02, 23 Июль 2023',
 'resource': 'https://lenta.ru'}
{'_id': ObjectId('64bcd2474155afe95155fedb'),
 'link': 'https://lenta.ru/news/2023/07/23/nepomenyay/',
 'name': 'Россиянин-чемпион ОИ по плаванию высказался о возможной смене '
         'гражданства',
 'news_time': '10:00, 23 Июль 2023',
 'resource': 'https://lenta.ru'}
{'_id': ObjectId('64bcd2474155afe95155fedc'),
 'link': 'https://moslenta.ru/news/city/muzhchina-vzorval-granatu-23-07-2023.htm/',
 'name': 'Мужчина взорвал гранату во время застолья в подмосковной деревн